## import 

In [1]:
import pandas as pd
import numpy as np
from kite_trade import *
from enctoken import get_kite
kite = get_kite()
import warnings
warnings.filterwarnings(action = 'ignore')
import time
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import clear_output

In [2]:
expiry_day = 'BANKNIFTY23810'
lower_price = 44000
upper_price = 46000

inst = pd.DataFrame(kite.instruments("NFO"))
expiry_march = (
    inst[(inst.name == "BANKNIFTY") & (inst.tradingsymbol.str.contains(expiry_day))]
    .query("(strike >= @lower_price )&( strike <= @upper_price)")
    .assign(strike_type=lambda x: x["strike"].astype(int).astype(str) + x["instrument_type"])
)

In [3]:
expiry_march.shape

(42, 13)

In [9]:
def get_data_minute(expiry_march, start_dt,end_dt,time_frame,bn_token,hist_data_path = None):
    df_combined = pd.DataFrame()
    strike_symbol_dict = dict(zip(expiry_march.instrument_token,expiry_march['strike_type'] ))
    strike_symbol_dict[bn_token] = "BANKNIFTY"
    inst_expiry = expiry_march['instrument_token'].unique().tolist()
    if expiry_march.shape[0] != 0:
        for i in inst_expiry:
            df_combined = df_combined.append(
            pd.DataFrame(
                kite.historical_data(
                    i,
                    from_date=start_dt,
                    to_date=end_dt,
                    interval=time_frame,
                    continuous=False,
                    oi=True,
                )
            ).assign(strike_type=strike_symbol_dict[i])
        )
    else:
        df = (
            pd.read_parquet(hist_data_path)
            .assign(date_only=lambda x: pd.to_datetime(x['date'].dt.date))
            .pipe(lambda x: x[x['date_only'] == pd.to_datetime(start_dt)])
            .assign(strike=lambda x: x['strike_type'].str[:5].astype(int))
            .query("(strike >= @lower_price) & (strike <= @upper_price)")
        )

    df_bn_hist_data = (
        pd.DataFrame(
            kite.historical_data(
                bn_token,
                from_date=start_dt,
                to_date=end_dt,
                interval=time_frame,
                continuous=False,
                oi=True,
            )
        )
        .assign(strike_type="BANKNIFTY")
        .drop("oi", axis=1)
        .assign(oi_change=lambda x: x["close"])
    )


    
    # df_bn_hist_data['oi_change'] = df_bn_hist_data.groupby(['strike_type']).oi.diff()

    df_combined = df_combined.assign(
        oi_change=lambda x: x.groupby("strike_type")["oi"].diff()
    ).assign(oi_change=lambda x: x.groupby("strike_type")["oi_change"].cumsum())


    df_combined_1 = (
        df_combined.dropna()
        .groupby("date")
        .agg({"oi_change": lambda x: sorted(x, reverse=True)[0:5]})
        .reset_index()
        .explode("oi_change")
        .merge(df_combined[['date','oi_change','strike_type']], how = 'left',on = ['date','oi_change'])
    )

    req_cols = df_combined_1.columns
    df_combined_1 = df_combined_1.append(df_bn_hist_data[req_cols]).rename(columns = {'oi_change':'oi'})
    # df_combined = df_combined.drop('oi',axis=1).append(df_bn_hist_data[req_cols]).rename(columns = {'oi_change':'oi'})
    return df_combined,df_combined_1,df_bn_hist_data

## strike price chg with BN

In [10]:
import time
import xlwings
## strike price change
start_dt = '2023-08-09'
end_dt = '2023-08-09'
time_frame = '5minute'
bn_token  = 260105
# hist_data_path = "../data/expiry_20April_5min.parquet"
hist_data_path = None
while True:
    df_combined,df_combined_1,df_bn_hist_data = get_data_minute(expiry_march, start_dt,end_dt,time_frame,bn_token,hist_data_path)
    break
    df_combined['date'] =  pd.to_datetime(df_combined['date']).dt.tz_localize(None)
    df_combined['close_change'] = df_combined.groupby(['strike_type']).close.diff()
    df_combined['strike'] = df_combined['strike_type'].str[5:]
    df_combined['strike_type'] = df_combined['strike_type'].str[:5]

    df_bn_hist_data["date"] = pd.to_datetime(df_bn_hist_data["date"]).dt.tz_localize(None)
    df_bn_hist_data["close_change"] = df_bn_hist_data.groupby(["strike_type"]).close.diff()

    df_combined = df_combined.astype({'strike_type':int}).query('strike_type>=44700 and strike_type <= 45300')
    df_combined = df_combined.append(df_bn_hist_data).assign(
        strike=lambda x: x["strike"].fillna("CE")
    )
    df_price_chg = pd.pivot(
        df_combined,
        index=["strike_type", "strike"],
        columns=["date"],
        values=["close_change"],
    )
    df_bn_pivot = pd.pivot(
        df_combined.query("strike_type =='BANKNIFTY' "),
        index=["strike_type", "strike"],
        columns=["date"],
        values=["close"],
    )

    df_bn_pivot.reset_index(inplace = True)
    df_bn_pivot.columns = [i[0] + '_'+ str(i[1]) for i in df_bn_pivot.columns]

    df_price_chg.reset_index(inplace = True)
    df_price_chg.columns = [i[0] + '_'+ str(i[1]) for i in df_price_chg.columns]
    df_price_chg.columns = [i.replace('close_change','close')for i in df_price_chg.columns ]
    df_price_chg.sort_values('strike_NaT',ascending = False, inplace = True)
    df_appended = df_bn_pivot.append(df_price_chg)

    
    # write dataframe df_appended in excel using xlwings
    # wb = xlwings.Book()
    wb = xlwings.Book(r'D:\US\McD-Discount-Portal\mcdds\notebooks\akash_dev\test.xlsx')
    sht = wb.sheets['Sheet4']
    sht.range('A1').value = df_appended
    time.sleep(30)
    # break

## OI change in excel

In [11]:
df_bn_hist_data['date'] = df_bn_hist_data['date'].dt.tz_localize(None)
df_combined['date'] = df_combined['date'].dt.tz_localize(None)

df_bn_hist_data['oi'] = df_bn_hist_data['close']
df_bn_hist_data['strike'] = 0
df_bn_hist_data['strike_type'] = "0"
df_combined = df_combined.append(df_bn_hist_data)

## geting pct change
d1 = df_combined.set_index(['date', 'strike_type']).oi.unstack(['strike_type'])
d1 = d1.pct_change().stack([0]).reset_index()
df_combined = df_combined.merge(d1, on=['date', 'strike_type'], how='left')
df_combined.rename(columns={0: 'pct'}, inplace=True)

# pivot 
df_combined['strike_price'] = df_combined.strike_type.str[:5].astype(int)
df_combined['side'] = df_combined.strike_type.str[5:]
df_combined.side = np.where(df_combined.side == '', 'CE-PE', df_combined.side)
df_combined.side = df_combined.side.str.split('-')
df_combined = df_combined.explode('side')

df_pivot = pd.pivot_table(df_combined,index = 'strike_price', columns=['date','side'], values='pct')
df_pivot.to_clipboard()

In [34]:
# df_combined = df_combined.merge(
#     df_combined.groupby(['date','side']).oi.sum()
#     .reset_index()
#     .rename(columns = {'oi': "oi_sum"}), how = 'left', on = ['date','side'])
# df_combined['contribution'] = (df_combined['oi'] / df_combined['oi_sum']) * df_combined['pct']
# df_cont = (df_combined
# .groupby(['date','side']).contribution.sum().reset_index().rename(columns = {'contribution': "pct"})
# .assign(strike_price = 999999))
# df_combined = df_combined.append(df_cont)

## plot oi chg

In [5]:
start_dt = '2023-05-24'
end_dt = '2023-05-24'
time_frame = '5minute'
bn_token  = 260105

while(True):
    hist_data_path = None
    df_combined,df_combined_1,df_bn_hist_data = get_data_minute(expiry_march, start_dt,end_dt,time_frame,bn_token,hist_data_path)
    print('run')
    df_temp = df_bn_hist_data.copy()
    for t in df_combined_1['date'].unique()[-2:]:
        # selecting top 5 sp
        df_current_time = df_combined_1.query('date == @t')
        st_pr = df_current_time['strike_type'].unique()

        # getting all prev OI of top 5 
        df_current_time_all = df_combined[['date','oi','strike_type']].query('(date <= @t)')
        df_bn_hist_data = df_temp.copy()
        df_bn_hist_data = df_bn_hist_data.query('(date <= @t)')
        df_current_time_all = df_current_time_all[df_current_time_all['strike_type'].isin(st_pr)]
        # df_current_time_all = df_current_time_all[~((df_current_time_all.strike_type == 'BANKNIFTY') & (df_current_time_all.oi == 0))]
        df_current_time_all.sort_values(['date','oi'], ascending=False, inplace = True)
        sp = df_current_time_all.strike_type.unique()
        
        fig = go.Figure()
        data = []
        for i in range(0,5):
            data.append(go.Scatter(x=df_current_time_all.query("strike_type == @sp[@i]")['date'],
                                        y=df_current_time_all.query("strike_type == @sp[@i]")['oi'],
                                        name = sp[i],
                                        yaxis='y1'
                                        ))

        # data.append(go.Scatter(x=df_current_time_all.query("strike_type == @sp[5]")['date'],
        #                             y=df_current_time_all.query("strike_type == @sp[5]")['oi'],
        #                             name = sp[5],
        #                             yaxis='y2'
        #                             ))

        data.append(go.Candlestick(x=df_bn_hist_data.date,
                open=df_bn_hist_data.open,
                high=df_bn_hist_data.high,
                low=df_bn_hist_data.low,
                close=df_bn_hist_data.close,
                name = "BN",
                    yaxis='y2'    
                            ))
        layout = go.Layout(title='OI vs Bank',
                        yaxis=dict(title='Oi',),
                        yaxis2=dict(title='bank',
                                    overlaying='y',
                                    side='right',
                                    tickformat = '0.0f', 
                                    #range =[39000,44000]
                                      )
                        )
                                
                                    
        
        fig = go.Figure(data=data, layout=layout)
        fig.update_layout(height=600, width=1200)
        fig.show()
        clear_output(wait=True)   
        time.sleep(300)
        # break|
    # break


run
